In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
from pathlib import Path
import logging
from skill_library.logging import file_logging_handler

# Add the src directory to sys.path
path = str(Path().resolve().parent)
if path not in sys.path:
    sys.path.insert(0, path)

print(sys.path)

# load_dotenv()
# print(str(dict(os.environ.items())))

# Set up structured logging to a file. All of the cells in this notebook use
# this logger. Find them at .data/logs.jsonl.

# Mute some noisy loggers.
modules = ['httpcore.connection', 'httpcore.http11', 'httpcore.sync.connection', 'httpx', 'openai', 'urllib3.connectionpool', 'urllib3.util.retry']
for module in modules:
    logging.getLogger(module).setLevel(logging.ERROR)

# Set up the file logging.
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers.clear()
file_handler = file_logging_handler(Path('.data') / 'logs.jsonl')
logger.addHandler(file_handler)


In [ ]:
%reload_ext autoreload
%autoreload 2

from pathlib import Path
import asyncio
import os

from assistant_drive import Drive, DriveConfig
from openai_client import AzureOpenAIAzureIdentityAuthConfig, AzureOpenAIServiceConfig, create_client
from openai_client.chat_driver import ChatDriverConfig
from pydantic_core import Url
from skill_library import Skill, Assistant

from guided_conversation_skill import GuidedConversationSkill
from guided_conversation_skill.conversation_guides.acrostic_poem import definition

azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT", "https://lightspeed-team-shared-openai-eastus.openai.azure.com/")
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")

service_config = AzureOpenAIServiceConfig(
    auth_config=AzureOpenAIAzureIdentityAuthConfig(),
    azure_openai_endpoint=Url(azure_openai_endpoint),
    azure_openai_deployment=azure_openai_deployment,
)
language_model = create_client(service_config)


# Configure the skill and its dependencies.
path = Path(".data/assistant-1")
drive = Drive(DriveConfig(root=path))
definition.resource_constraint.quantity = 5  # type: ignore
skill = GuidedConversationSkill(name="guided_conversation", language_model=language_model, definition=definition, drive=drive.subdrive("gc1"))
skill_dependency_tree: dict[str, Skill] = {skill.name: skill}

# Define the assistant.
chat_driver_config = ChatDriverConfig(
    openai_client=language_model,
    model="gpt-4o",
    instructions="You are a helpful assistant.",
)

assistant = Assistant(name="Alice", assistant_id="gc-assistant-123", chat_driver_config=chat_driver_config, skills=skill_dependency_tree, startup_routine="guided_conversation.conversation")
await assistant.clear()
await assistant.start()


# Function that allows user input in a non-blocking manner.
async def user_input_handler() -> None:
    while True:
        user_input = await asyncio.to_thread(input, "User: ")
        if user_input == "":
            assistant.stop()
            break
        print(f"User: {user_input}", flush=True)
        await assistant.put_message(user_input)


# Start the user input in a non-blocking way.
input_task = asyncio.create_task(user_input_handler())

# Start the assistant.
async for event in assistant.events:
    print(f"Assistant: {event.message}", flush=True)  # type: ignore
    if event.message == "Conversation complete!":
        print()
        break

await assistant.wait()
await input_task

